In [1]:
import os, sys
cur_dir = os.getcwd()
basename = os.path.basename(cur_dir)
for _ in range(5):
    if basename != 'OCLog':
        cur_dir = os.path.dirname(cur_dir)
        basename = os.path.basename(cur_dir)
        #print(cur_dir, basename)
    else:
        if cur_dir not in sys.path:
            sys.path.append(cur_dir)
            #print(sys.path)
import os
from openpyxl import Workbook
from openpyxl import load_workbook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tqdm import trange, tqdm, tnrange
from oclog.BGL.bgUKCNum import BGLog, get_embedding_layer
from oclog.openset.boundary_loss import euclidean_metric, BoundaryLoss
from oclog.openset.pretraining import LogLineEncoder, LogSeqEncoder, LogClassifier
# from oclog.openset.openset import OpenSet
from oclog.openset.opensetv7 import OpenSet
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
import sklearn.metrics as m
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.manifold import TSNE

In [ ]:
designated_ukc_cls = 5
embedding_size = 12
num_classes= 3
pretrain_epochs = 6
ukc_label=7
bglog = BGLog(save_padded_num_sequences=False, load_from_pkl=True, )
train_test = bglog.get_tensor_train_val_test(ablation=2500, designated_ukc_cls=designated_ukc_cls)
train_data, val_data,  test_data = train_test
print('length of train_data:', len(train_data))
print('length of val_data:',len(val_data))
line_encoder = LogLineEncoder(bglog, chars_in_line=64)
logSeqencer =  LogSeqEncoder(line_in_seq=32, dense_neurons=embedding_size)
ptmodel = LogClassifier(line_encoder=line_encoder, seq_encoder=logSeqencer, num_classes=num_classes)
ptmodel.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
hist = ptmodel.fit(train_data, validation_data=val_data, epochs=pretrain_epochs)    

octrain_epochs = 50
lr_rate = 2.5
wait_patience = 8
optimizer = 'adam'

oset = OpenSet(num_classes, ptmodel, embedding_size=embedding_size, ukc_label=ukc_label)
_, _ = oset.train(train_data, data_val=val_data, data_test=test_data, epochs=octrain_epochs, 
                  lr_rate=lr_rate, wait_patience=wait_patience, optimizer=optimizer,
                 pretrain_hist=hist)

padded_num_seq_df loaded from C:\Users\Bhujay_ROG\MyDev\OCLog\oclog\BGL\data\bgl_ukc.pkl
trained tokenizer, tk, loaded from C:\Users\Bhujay_ROG\MyDev\OCLog\oclog\BGL\data\bgl_tk.pkl
train_0:, 2000, val_0:, 250, test_0:, 250, train_1:, 2000, val_1:, 250, test_1:, 250, train_2:, 2000, val_2:, 250, test_2:, 250, class 5 is added as ukc
ukc_5:, 165
length of train_data - (num_seq_per_cls * num_class)// batch size: 187
length of train_data: 187
length of val_data: 23
vocab_size: 50
Epoch 1/6
187/187 [==============================] - 19s 95ms/step - loss: 0.0702 - accuracy: 0.9741 - precision: 0.9872 - recall: 0.9564 - val_loss: 1.9167e-04 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 2/6
187/187 [==============================] - 18s 94ms/step - loss: 0.0030 - accuracy: 0.9993 - precision: 0.9995 - recall: 0.9993 - val_loss: 1.1778e-04 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 3/6
187/187 [==============================] - 18s 94m

In [ ]:
# oset = OpenSet(num_classes, ptmodel, embedding_size=embedding_size, ukc_label=ukc_label)

In [ ]:
 _, _, f1_weighted, f_measure = oset.evaluate(test_data, ukc_label=designated_ukc_cls, store_features=True)

In [ ]:
a = np.array(oset.total_features)
print(a.shape)
# a = np.reshape(a, ((a.shape[0] * a.shape[1]), a.shape[2])    )
# print(a.shape)
p = np.array(oset.total_preds)
print(p.shape)
lbl_a = zip(a, p)
lbl_a = np.array(lbl_a)
print(lbl_a.shape)

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE()
tout = tsne.fit_transform(a)
print('tout.shape', tout.shape,  'a',a[0], 'tout', tout[0], 'label', p)
plt.scatter(tout[:, 0], tout[:, -1], c=p, s=20, cmap='tab10')
plt.show()

In [ ]:
plt.scatter(tout[:, 0], tout[:, -1], c=p, s=10, cmap='tab10')

In [ ]:
c = oset.centroids.numpy()
print('c:', c.shape)
cout = tsne.fit_transform(c)
print(cout[0])
plt.scatter(cout[:, 0], cout[:, -1], c=[0, 1, 9], s=400, cmap='tab10', marker=r'd', edgecolors= 'k')

In [ ]:
# perplexity=50, max_iter=2000, early_exag_coeff=12, stop_lying_iter=1000

In [ ]:
tsne = TSNE(perplexity=200,  )
tout = tsne.fit_transform(a)
print('tout.shape', tout.shape,  'a',a[0], 'tout', tout[0], 'label', p)
scatter = plt.scatter(tout[:, 0], tout[:, -1], c=p, s=20, cmap='tab10', label=p)
plt.show()

In [ ]:
# sns.lmplot(x=tout[:, 0], y=tout[:, -1], data=tout)

In [ ]:
fig, ax = plt.subplots()
scatter = ax.scatter(tout[:, 0], tout[:, -1], c=p, s=20, cmap='tab10',)
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="upper left", title="Classes")
ax.add_artist(legend1)
ax.scatter(cout[:, 0], cout[:, -1], c=[0, 1, 2], s=20, cmap='tab10', marker=r'd', edgecolors= 'k')
# handles, labels = scatter.legend_elements(prop="sizes", alpha=0.6)
# legend2 = ax.legend(handles, labels, loc="upper right", title="Sizes")

plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScalar
scaler = MinMaxScaler()
scaled_tout = scaled = scaler.fit_transform(tout)
scaled_cout = scaled = scaler.fit_transform(cout)

In [ ]:
fig, ax = plt.subplots()
scatter = ax.scatter(scaled_tout[:, 0], scaled_tout[:, -1], c=p, s=20, cmap='tab10',)
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="upper left", title="Classes")
ax.add_artist(legend1)
ax.scatter(scaled_cout[:, 0], scaled_cout[:, -1], c=[0, 1, 2], s=200, cmap='tab10', marker=r'd', edgecolors= 'k')

In [ ]:
np.unique(p)

In [ ]:
# ['Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
#                       'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
#                       'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn'])
# all_colors=['Greens', 'Oranges', 'Greys', 'YlOrBr', 'Blues', 'YlGnBu', 'PuBu', 'Reds']


#### color automatically generated
fig, ax = plt.subplots()
scatter = ax.scatter(scaled_tout[:, 0], scaled_tout[:, -1], c=p, s=20, cmap='tab10',)
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="upper left", title="Classes")
ax.add_artist(legend1)
cmap_1 = scatter.get_cmap().colors
### get the color of the feature scatter and use it in the centroid scatter
ccolor = [cmap_1[i] for i in  range(len(c))]
print(ccolor)
ax.scatter(scaled_cout[:, 0], scaled_cout[:, -1], c=ccolor, s=200, cmap='tab10', marker=r'd', edgecolors= 'k')

In [ ]:
color_maps = np.array(["green","blue","yellow","pink","black","orange","purple","red","beige","brown","gray","cyan","magenta"])
pcolor = [all_colors[i] for i in  p]
pcolor = np.array(pcolor)
# ccolor = [all_colors[i] for i in  range(len(c))]
fig, ax = plt.subplots()
scatter = ax.scatter(scaled_tout[:, 0], scaled_tout[:, -1], c=color_maps[p], s=20, cmap='tab10',)
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="upper left", title="Classes")
ax.add_artist(legend1)
cmap_1 = scatter.get_cmap().colors
ccolor = np.array([i for i in  range(len(c))])
print(ccolor)
ax.scatter(scaled_cout[:, 0], scaled_cout[:, -1], c=color_maps[ccolor], s=200, cmap='tab10', marker=r'd', edgecolors= 'k')

In [ ]:
x = np.arange(1, 2, 3, 4)
y = np.arange(1, 4, 6, 8,)

color_maps = np.array(["red","green","blue","yellow","pink","black","orange","purple","beige","brown","gray","cyan","magenta"])
# colors = np.array(["red","green","blue",])
categories = []
pcolor = color_maps[]

print(x, y, colors)
# plt.scatter(x, y , c=colors)

In [ ]:
a = np.array(oset.total_features)
c = oset.centroids.numpy()
tsne = TSNE()
tout = tsne.fit_transform(a)
cout = tsne.fit_transform(c)
scaler = MinMaxScaler()
scaled_tout = scaler.fit_transform(tout)
scaled_cout = scaler.fit_transform(cout)
color_maps = np.array(["green","blue","yellow","pink","black","orange","purple","red","beige","brown","gray","cyan","magenta"])
pcolor = [all_colors[i] for i in  p]
pcolor = np.array(pcolor)
# ccolor = [all_colors[i] for i in  range(len(c))]
fig, ax = plt.subplots()
scatter = ax.scatter(scaled_tout[:, 0], scaled_tout[:, -1], c=color_maps[p], s=20, cmap='tab10',)
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="upper left", title="Classes",  bbox_to_anchor=(1.05, 1))
ax.add_artist(legend1)
cmap_1 = scatter.get_cmap().colors
ccolor = np.array([i for i in  range(len(c))])
print(ccolor)
ax.scatter(scaled_cout[:, 0], scaled_cout[:, -1], c=color_maps[ccolor], s=200, cmap='tab10', marker=r'd', edgecolors= 'k')

In [ ]:
a = np.array(oset.total_features)
c = oset.centroids.numpy()
tsne = TSNE(perplexity=300,)
tout = tsne.fit_transform(a)
cout = tsne.fit_transform(c)
scaler = MinMaxScaler()
scaled_tout = scaler.fit_transform(tout)
scaled_cout = scaler.fit_transform(cout)
color_maps = np.array(["green","blue","yellow","pink","black","orange","purple","red","beige","brown","gray","cyan","magenta"])
pcolor = [all_colors[i] for i in  p]
pcolor = np.array(pcolor)
# ccolor = [all_colors[i] for i in  range(len(c))]
fig, ax = plt.subplots()
scatter = ax.scatter(scaled_tout[:, 0], scaled_tout[:, -1], c=color_maps[p], s=20, cmap='tab10',)
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="upper left", title="Classes",  bbox_to_anchor=(1.05, 1))
ax.add_artist(legend1)
cmap_1 = scatter.get_cmap().colors
ccolor = np.array([i for i in  range(len(c))])
print(ccolor)
ax.scatter(scaled_cout[:, 0], scaled_cout[:, -1], c=color_maps[ccolor], s=200, cmap='tab10', marker=r'd', edgecolors= 'k')

In [ ]:
a = np.array(oset.total_features)
c = oset.centroids.numpy()
tsne = TSNE(perplexity=300,)
tout = tsne.fit_transform(a)
cout = tsne.fit_transform(c)
scaler = MinMaxScaler()
scaled_tout = scaler.fit_transform(tout)
scaled_cout = scaler.fit_transform(cout)
color_maps = np.array(["green","blue","yellow","pink","black","orange","purple","red","beige","brown","gray","cyan","magenta"])
pcolor = [all_colors[i] for i in  p]
pcolor = np.array(pcolor)
# ccolor = [all_colors[i] for i in  range(len(c))]
fig, ax = plt.subplots()
scatter = ax.scatter(scaled_tout[:, 0], scaled_tout[:, -1], c=color_maps[p], s=20, cmap='tab10',)
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="upper left", title="Classes",  bbox_to_anchor=(1.05, 1))
ax.add_artist(legend1)
cmap_1 = scatter.get_cmap().colors
ccolor = np.array([i for i in  range(len(c))])
print(ccolor)
ax.scatter(scaled_cout[:, 0], scaled_cout[:, -1],  s=200, cmap='tab10', marker=r'd', edgecolors= 'k')

In [ ]:
a = np.array(oset.total_features)
c = oset.centroids.numpy()
tsne = TSNE(perplexity=10,)
tout = tsne.fit_transform(a)
cout = tsne.fit_transform(c)
scaler = MinMaxScaler()
scaled_tout = scaler.fit_transform(tout)
scaled_cout = scaler.fit_transform(cout)
color_maps = np.array(["green","blue","yellow","pink","black","orange","purple","red","beige","brown","gray","cyan","magenta"])
pcolor = [all_colors[i] for i in  p]
pcolor = np.array(pcolor)
# ccolor = [all_colors[i] for i in  range(len(c))]
fig, ax = plt.subplots()
scatter = ax.scatter(scaled_tout[:, 0], scaled_tout[:, -1], c=color_maps[p], s=20, cmap='tab10',)
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="upper left", title="Classes",  bbox_to_anchor=(1.05, 1))
ax.add_artist(legend1)
cmap_1 = scatter.get_cmap().colors
ccolor = np.array([i for i in  range(len(c))])
print(ccolor)
ax.scatter(scaled_cout[:, 0], scaled_cout[:, -1],  s=200, c=color_maps[ccolor], cmap='tab10', marker=r'd', edgecolors= 'k')

In [ ]:
a = np.array(oset.total_features)
c = oset.centroids.numpy()
tsne = TSNE(perplexity=5,)
tout = tsne.fit_transform(a)
cout = tsne.fit_transform(c)
scaler = MinMaxScaler()
scaled_tout = scaler.fit_transform(tout)
scaled_cout = scaler.fit_transform(cout)
color_maps = np.array(["green","blue","yellow","pink","black","orange","purple","red","beige","brown","gray","cyan","magenta"])
pcolor = [all_colors[i] for i in  p]
pcolor = np.array(pcolor)
# ccolor = [all_colors[i] for i in  range(len(c))]
fig, ax = plt.subplots()
scatter = ax.scatter(scaled_tout[:, 0], scaled_tout[:, -1], c=color_maps[p], s=20, cmap='tab10',)
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="upper left", title="Classes",  bbox_to_anchor=(1.05, 1))
ax.add_artist(legend1)
cmap_1 = scatter.get_cmap().colors
ccolor = np.array([i for i in  range(len(c))])
print(ccolor)
ax.scatter(scaled_cout[:, 0], scaled_cout[:, -1],  s=200, c=color_maps[ccolor], cmap='tab10', marker=r'd', edgecolors= 'k')

In [ ]:
a = np.array(oset.total_features)
c = oset.centroids.numpy()
tsne = TSNE(perplexity=1000,)
tout = tsne.fit_transform(a)
cout = tsne.fit_transform(c)
scaler = MinMaxScaler()
scaled_tout = scaler.fit_transform(tout)
scaled_cout = scaler.fit_transform(cout)
color_maps = np.array(["green","blue","yellow","pink","black","orange","purple","red","beige","brown","gray","cyan","magenta"])
pcolor = [all_colors[i] for i in  p]
pcolor = np.array(pcolor)
# ccolor = [all_colors[i] for i in  range(len(c))]
fig, ax = plt.subplots()
scatter = ax.scatter(scaled_tout[:, 0], scaled_tout[:, -1], c=color_maps[p], s=20, cmap='tab10',)
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="upper left", title="Classes",  bbox_to_anchor=(1.05, 1))
ax.add_artist(legend1)
cmap_1 = scatter.get_cmap().colors
ccolor = np.array([i for i in  range(len(c))])
print(ccolor)
ax.scatter(scaled_cout[:, 0], scaled_cout[:, -1],  s=200, c=color_maps[ccolor], cmap='tab10', marker=r'd', edgecolors= 'k')

In [ ]:
a = np.array(oset.total_features)
c = oset.centroids.numpy()
tsne = TSNE(perplexity=50,)
tout = tsne.fit_transform(a)
cout = tsne.fit_transform(c)
scaler = MinMaxScaler()
scaled_tout = scaler.fit_transform(tout)
scaled_cout = scaler.fit_transform(cout)
color_maps = np.array(["green","blue","yellow","pink","black","orange","purple","red","beige","brown","gray","cyan","magenta"])
pcolor = [all_colors[i] for i in  p]
pcolor = np.array(pcolor)
# ccolor = [all_colors[i] for i in  range(len(c))]
fig, ax = plt.subplots()
scatter = ax.scatter(scaled_tout[:, 0], scaled_tout[:, -1], c=color_maps[p], s=20, cmap='tab10',)
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="upper left", title="Classes",  bbox_to_anchor=(1.05, 1))
ax.add_artist(legend1)
cmap_1 = scatter.get_cmap().colors
ccolor = np.array([i for i in  range(len(c))])
print(ccolor)
ax.scatter(scaled_cout[:, 0], scaled_cout[:, -1],  s=200, c=color_maps[ccolor], cmap='tab10', marker=r'd', edgecolors= 'k')

In [ ]:
a = np.array(oset.total_features)
c = oset.centroids.numpy()
tsne = TSNE(perplexity=150,)
tout = tsne.fit_transform(a)
cout = tsne.fit_transform(c)
scaler = MinMaxScaler()
scaled_tout = scaler.fit_transform(tout)
scaled_cout = scaler.fit_transform(cout)
color_maps = np.array(["green","blue","yellow","pink","black","orange","purple","red","beige","brown","gray","cyan","magenta"])
pcolor = [all_colors[i] for i in  p]
pcolor = np.array(pcolor)
# ccolor = [all_colors[i] for i in  range(len(c))]
fig, ax = plt.subplots()
scatter = ax.scatter(scaled_tout[:, 0], scaled_tout[:, -1], c=color_maps[p], s=20, cmap='tab10',)
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="upper left", title="Classes",  bbox_to_anchor=(1.05, 1))
ax.add_artist(legend1)
cmap_1 = scatter.get_cmap().colors
ccolor = np.array([i for i in  range(len(c))])
print(ccolor)
ax.scatter(scaled_cout[:, 0], scaled_cout[:, -1],  s=200, c=color_maps[ccolor], cmap='tab10', marker=r'd', edgecolors= 'k')

In [ ]:
a = np.array(oset.total_features)
c = oset.centroids.numpy() // 5
tsne = TSNE(perplexity=150,)
tout = tsne.fit_transform(a)
cout = tsne.fit_transform(c)
scaler = MinMaxScaler()
scaled_tout = scaler.fit_transform(tout)
scaled_cout = scaler.fit_transform(cout)
color_maps = np.array(["green","blue","yellow","pink","black","orange","purple","red","beige","brown","gray","cyan","magenta"])
pcolor = [all_colors[i] for i in  p]
pcolor = np.array(pcolor)
# ccolor = [all_colors[i] for i in  range(len(c))]
fig, ax = plt.subplots()
scatter = ax.scatter(scaled_tout[:, 0], scaled_tout[:, -1], c=color_maps[p], s=20, cmap='tab10',)
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="upper left", title="Classes",  bbox_to_anchor=(1.05, 1))
ax.add_artist(legend1)
cmap_1 = scatter.get_cmap().colors
ccolor = np.array([i for i in  range(len(c))])
print(ccolor)
ax.scatter(scaled_cout[:, 0], scaled_cout[:, -1],  s=200, c=color_maps[ccolor], cmap='tab10', marker=r'd', edgecolors= 'k')

In [ ]:
####  ##

In [ ]:
a = np.array(oset.total_features)
c = oset.centroids.numpy()
tsne = TSNE(perplexity=150,)
tout = tsne.fit_transform(a)
cout = tsne.fit_transform(c)
m_scaler = MinMaxScaler()
s_scalar = StandardScaler

scaled_tout = m_scaler.fit_transform(tout)
scaled_cout = m_scaler.fit_transform(cout)

# scaled_tout = s_scalar.fit_transform(tout)
# scaled_cout = s_scalar.fit_transform(cout)
color_maps = np.array(["green","blue","yellow","pink","black","orange","purple","red","beige","brown","gray","cyan","magenta"])
pcolor = [all_colors[i] for i in  p]
pcolor = np.array(pcolor)
# ccolor = [all_colors[i] for i in  range(len(c))]
fig, ax = plt.subplots()
scatter = ax.scatter(scaled_tout[:, 0], scaled_tout[:, -1], c=color_maps[p], s=20, cmap='tab10',)
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="upper left", title="Classes",  bbox_to_anchor=(1.05, 1))
ax.add_artist(legend1)
cmap_1 = scatter.get_cmap().colors
ccolor = np.array([i for i in  range(len(c))])
print(ccolor)
ax.scatter(scaled_cout[:, 0], scaled_cout[:, -1],  s=200, c=color_maps[ccolor], cmap='tab10', marker=r'd', edgecolors= 'k')

In [ ]:
a = np.array(oset.total_features)
c = oset.centroids.numpy()
tsne = TSNE(perplexity=150,)
tout = tsne.fit_transform(a)
cout = tsne.fit_transform(c)
m_scaler = MinMaxScaler()
s_scalar = StandardScaler()

# scaled_tout = m_scaler.fit_transform(tout)
# scaled_cout = m_scaler.fit_transform(cout)

scaled_tout = s_scalar.fit_transform(tout)
scaled_cout = s_scalar.fit_transform(cout)
color_maps = np.array(["green","blue","yellow","pink","black","orange","purple","red","beige","brown","gray","cyan","magenta"])
pcolor = [all_colors[i] for i in  p]
pcolor = np.array(pcolor)
# ccolor = [all_colors[i] for i in  range(len(c))]
fig, ax = plt.subplots()
scatter = ax.scatter(scaled_tout[:, 0], scaled_tout[:, -1], c=color_maps[p], s=20, cmap='tab10',)
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="upper left", title="Classes",  bbox_to_anchor=(1.05, 1))
ax.add_artist(legend1)
cmap_1 = scatter.get_cmap().colors
ccolor = np.array([i for i in  range(len(c))])
print(ccolor)
ax.scatter(scaled_cout[:, 0], scaled_cout[:, -1],  s=200, c=color_maps[ccolor], cmap='tab10', marker=r'd', edgecolors= 'k')

In [ ]:
a = np.array(oset.total_features)
c = oset.centroids.numpy()
tsne = TSNE(perplexity=100,)
tout = tsne.fit_transform(a)
cout = tsne.fit_transform(c)
m_scaler = MinMaxScaler()
s_scalar = StandardScaler()

# scaled_tout = m_scaler.fit_transform(tout)
# scaled_cout = m_scaler.fit_transform(cout)

scaled_tout = s_scalar.fit_transform(tout)
scaled_cout = s_scalar.fit_transform(cout)
color_maps = np.array(["green","blue","yellow","pink","black","orange","purple","red","beige","brown","gray","cyan","magenta"])
pcolor = [all_colors[i] for i in  p]
pcolor = np.array(pcolor)
# ccolor = [all_colors[i] for i in  range(len(c))]
fig, ax = plt.subplots()
scatter = ax.scatter(scaled_tout[:, 0], scaled_tout[:, -1], c=color_maps[p], s=20, cmap='tab10',)
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="upper left", title="Classes",  bbox_to_anchor=(1.05, 1))
ax.add_artist(legend1)
cmap_1 = scatter.get_cmap().colors
ccolor = np.array([i for i in  range(len(c))])
print(ccolor)
ax.scatter(scaled_cout[:, 0], scaled_cout[:, -1],  s=200, c=color_maps[ccolor], cmap='tab10', marker=r'd', edgecolors= 'k')

In [ ]:
a = np.array(oset.total_features)
c = oset.centroids.numpy()
tsne = TSNE(perplexity=150,)
tout = tsne.fit_transform(a)
cout = tsne.fit_transform(c)
m_scaler = MinMaxScaler()
s_scalar = StandardScaler()

# scaled_tout = m_scaler.fit_transform(tout)
# scaled_cout = m_scaler.fit_transform(cout)

scaled_tout = s_scalar.fit_transform(tout)
scaled_cout = s_scalar.fit_transform(cout)
color_maps = np.array(["green","blue","yellow","pink","black","orange","purple","red","beige","brown","gray","cyan","magenta"])
pcolor = [all_colors[i] for i in  p]
pcolor = np.array(pcolor)
# ccolor = [all_colors[i] for i in  range(len(c))]
fig, ax = plt.subplots()
scatter = ax.scatter(scaled_tout[:, 0], scaled_tout[:, -1], c=color_maps[p], s=20, cmap='tab10',)
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="upper left", title="Classes",  bbox_to_anchor=(1.05, 1))
ax.add_artist(legend1)
cmap_1 = scatter.get_cmap().colors
ccolor = np.array([i for i in  range(len(c))])
print(ccolor)
ax.scatter(scaled_cout[:, 0], scaled_cout[:, -1],  s=200, c=color_maps[ccolor], cmap='tab10', marker=r'd', edgecolors= 'k')

In [ ]:
a = np.array(oset.total_features)
c = oset.centroids.numpy()
tsne = TSNE(perplexity=200, early_exaggeration=12, random_state=123, learning_rate=80)
tout = tsne.fit_transform(a)
cout = tsne.fit_transform(c)
m_scaler = MinMaxScaler()
s_scalar = StandardScaler()

# scaled_tout = m_scaler.fit_transform(tout)
# scaled_cout = m_scaler.fit_transform(cout)

scaled_tout = s_scalar.fit_transform(tout)
scaled_cout = s_scalar.fit_transform(cout)
color_maps = np.array(["green","blue","yellow","pink","black","orange","purple","red","beige","brown","gray","cyan","magenta"])
pcolor = [all_colors[i] for i in  p]
pcolor = np.array(pcolor)
# ccolor = [all_colors[i] for i in  range(len(c))]
fig, ax = plt.subplots()
scatter = ax.scatter(scaled_tout[:, 0], scaled_tout[:, -1], c=color_maps[p], s=20, cmap='tab10',)
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="upper left", title="Classes",  bbox_to_anchor=(1.05, 1))
ax.add_artist(legend1)
cmap_1 = scatter.get_cmap().colors
ccolor = np.array([i for i in  range(len(c))])
print(ccolor)
ax.scatter(scaled_cout[:, 0], scaled_cout[:, -1],  s=200, c=color_maps[ccolor], cmap='tab10', marker=r'd', edgecolors= 'k')

In [ ]:
a = np.array(oset.total_features)
c = oset.centroids.numpy()
tsne = TSNE(perplexity=200, early_exaggeration=12, random_state=123, learning_rate=80)
tout = tsne.fit_transform(a)
cout = tsne.fit_transform(c)
m_scaler = MinMaxScaler()
s_scalar = StandardScaler()

# scaled_tout = m_scaler.fit_transform(tout)
# scaled_cout = m_scaler.fit_transform(cout)

scaled_tout = s_scalar.fit_transform(tout)
scaled_cout = s_scalar.fit_transform(cout)
color_maps = np.array(["green","blue","yellow","pink","black","orange","purple","red","beige","brown","gray","cyan","magenta"])
pcolor = [all_colors[i] for i in  p]
pcolor = np.array(pcolor)
# ccolor = [all_colors[i] for i in  range(len(c))]
fig, ax = plt.subplots()
scatter = ax.scatter(scaled_tout[:, 0], scaled_tout[:, -1], c=color_maps[p], s=20, cmap='tab10',)
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="upper left", title="Classes",  bbox_to_anchor=(1.05, 1))
ax.add_artist(legend1)
cmap_1 = scatter.get_cmap().colors
ccolor = np.array([i for i in  range(len(c))])
print(ccolor)
ax.scatter(scaled_cout[:, 0], scaled_cout[:, -1],  s=200, c=color_maps[ccolor], cmap='tab10', marker=r'd', edgecolors= 'k')

In [ ]:
a = np.array(oset.total_features)
c = oset.centroids.numpy()
tsne = TSNE(perplexity=200, early_exaggeration=12, random_state=123, learning_rate=80)
tout = tsne.fit_transform(a)
cout = tsne.fit_transform(c)
m_scaler = MinMaxScaler()
s_scalar = StandardScaler()

# scaled_tout = m_scaler.fit_transform(tout)
# scaled_cout = m_scaler.fit_transform(cout)

scaled_tout = s_scalar.fit_transform(tout)
scaled_cout = s_scalar.fit_transform(cout)
color_maps = np.array(["green","blue","yellow","pink","black","orange","purple","red","beige","brown","gray","cyan","magenta"])
pcolor = [all_colors[i] for i in  p]
pcolor = np.array(pcolor)
# ccolor = [all_colors[i] for i in  range(len(c))]
fig, ax = plt.subplots()
scatter = ax.scatter(scaled_tout[:, 0], scaled_tout[:, -1], c=color_maps[p], s=20, cmap='tab10',)
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="upper left", title="Classes",  bbox_to_anchor=(1.05, 1))
ax.add_artist(legend1)
cmap_1 = scatter.get_cmap().colors
ccolor = np.array([i for i in  range(len(c))])
print(ccolor)
ax.scatter(scaled_cout[:, 0], scaled_cout[:, -1],  s=200, c=color_maps[ccolor], cmap='tab10', marker=r'd', edgecolors= 'k')

In [ ]:
oset.centroids.numpy()/10

In [ ]:
fig, ax = plt.subplots()
scatter = ax.scatter(scaled_tout[:, 0], scaled_tout[:, -1], c=p, s=20, cmap='tab10', )
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="upper left", title="Classes",  bbox_to_anchor=(1.05, 1))
ax.add_artist(legend1)
cmap_1 = scatter.get_cmap().colors
ccolor = np.array([cmap_1[i] for i in  range(len(c))])
print(ccolor)
ax.scatter(scaled_cout[:, 0], scaled_cout[:, -1],  s=200, c=ccolor, cmap='tab10', marker=r'd', edgecolors= 'k')
ax.set_xlabel("class features and their centroids")
plt.show()

In [ ]:
scatter.legend_elements()

In [ ]:
plt.figure(figsize=(20, 20))
ax = plt.subplot(3, 2, 5)
fig5 = ax.scatter(scaled_tout[:, 0], scaled_tout[:, -1], c=p, s=20, cmap='tab10', )
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="upper left", title="Classes",  bbox_to_anchor=(1.05, 1))
ax.add_artist(legend1)
cmap_1 = scatter.get_cmap().colors
ccolor = np.array([cmap_1[i] for i in  range(len(c))])
ax.scatter(scaled_cout[:, 0], scaled_cout[:, -1],  s=200, c=ccolor, cmap='tab10', marker=r'd', edgecolors= 'k')
ax.set_xlabel("class features and their centroids")
plt.show()

In [ ]:
f1 = [0, 1, 0, 0, 0]
f2 = [0, 0, 2, 0, 0]
f3 = [0, 0, 3, 0, 0]
f4 = [0, 0, 4, 0, 0]
b1 = [f1, f2]
b2 = [f2, f3]

a = np.array([b1, b2])
print(a.shape)
print(a)
b = np.reshape(a, (4, 5))
print(b.shape)
print(b)

In [ ]:
len(train_data)